In [1]:
import cv2

def preprocess_image(image_path):
    # Baca gambar
    img = cv2.imread(image_path)
    # Resize gambar menjadi 100x100
    resized_img = cv2.resize(img, (100, 100))
    return resized_img


In [2]:
import cv2
import numpy as np

def extract_features(image):
    # Konversi gambar ke HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Ekstrak histogram warna
    hist = cv2.calcHist([hsv_img], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    # Normalisasi histogram
    cv2.normalize(hist, hist)
    return hist.flatten()

In [3]:
from sklearn.cluster import KMeans
import numpy as np

# Anggap kita sudah punya data fitur dari setiap gambar
features = []
for i in range(1, 11):
    img = preprocess_image(f'{i}.png')
    feature = extract_features(img)
    features.append(feature)

features = np.array(features)

# Tentukan jumlah cluster k (misalnya 3)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(features)

# Lihat hasil clustering
labels = kmeans.labels_
print(labels)  # Output label cluster dari setiap gambar


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [4]:
import matplotlib.pyplot as plt

for i in range(3):  # Misal ada 3 cluster
    cluster_images = [f'{j+1}.png' for j in range(10) if labels[j] == i]
    print(f"Cluster {i}: {cluster_images}")


NameError: name 'labels' is not defined